<h1 style='font-size:40px'> PySpark Tutorial</h1>
<h2 style='font-size:30px'>Instalação </h2>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 58 kB/s  eta 0:00:011   |█▋                              | 14.5 MB 168 kB/s eta 0:26:26     |██████████████████████▍         | 196.7 MB 362 kB/s eta 0:03:54     |█████████████████████████▎      | 221.9 MB 454 kB/s eta 0:02:11     |█████████████████████████████▉  | 262.7 MB 329 kB/s eta 0:00:57
     |████████████████████████████████| 198 kB 334 kB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=34c0c7b07e48c4f04eb7b4babbb5bcf6de26074fd7257fc2a15b7776a9e2ead0
  Stored in directory: /home/veiga/.cache/pip/wheels/58/94/83/915c9059e4b038e2d43a6058f307fe1c3e8536e5745f3b23b7
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


<div> 
    <hr>
    <h2 style='font-size:30px'> Introdução</h2>
</div>

In [28]:
# Por efeitos de comparação, vamos carregar um arquivo csv no pandas
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Pyspark-With-Python/main/test1.csv')
df.to_csv('test1.csv', columns=['Name', 'age', 'Experience', 'Salary'], index=False)

<div> 
    <ul style='font-size:20px'>
        <li>
            Iremos agora criar uma sessão no Spark. Para isso, é necessário importar o objeto SparkSession, de pyspark.sql.
        </li>
    </ul>
</div>
<strong style='font-size:16px'>Nota: para se usar o pyspark, é necessário ter o Java instalado em sua máquina! </strong>

In [10]:
# A seção deve ter um determinado nome.
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate() 
spark

In [29]:
# Lendo um arquivo csv. Note como a sintaxe é semelhante com a do pandas.
# O pyspark, por padrão, não considera a primeira linha do csv como o header. Para que isso aconteça, defina o argumento
# 'header', de 'read.csv' como 'true'.
df_pyspark = spark.read.csv('test1.csv', header='true')

# Logo de cara, já notamos a estética distinta dos DataFrames do pyspark.
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [33]:
# A biblioteca tem inúmeras semelhanças com o pandas.
df_pyspark.head(3)

[Row(Name='Krish', age='31', Experience='10', Salary='30000'),
 Row(Name='Sudhanshu', age='30', Experience='8', Salary='25000'),
 Row(Name='Sunny', age='29', Experience='4', Salary='20000')]

<p style='color:red'> Parei em 14:10</p>